In [ ]:
import sys
sys.path.insert(0,'c:/MyDocs/integrated/') # adjust to your setup

%run "catalog_support.py" 

Numdays = 4

showHeader('Raw Disclosures',line2=f'{Numdays} days of FracFocus changes',use_remote=True)

In [ ]:
import datetime

last_repo = datetime.datetime(year=2024,month=10,day=8)
today = datetime.datetime.today()
edate = today - datetime.timedelta(days=Numdays)
# print('earlist shown',edate)

daily_status_folder = r"G:\My Drive\webshare\daily_status"
added_disc_fn = os.path.join(daily_status_folder,'added_disc_df.parquet')
removed_disc_fn = os.path.join(daily_status_folder,'removed_disc_df.parquet')
modified_disc_fn = os.path.join(daily_status_folder,'modified_disc_df.parquet')


In [ ]:
import os
import pickle
import openFF.common.text_handlers as th
arc_dir = r"C:\MyDocs\integrated\openFF_archive\diff_dicts"

diff_fns = os.listdir(arc_dir)
download_dates = []
added = []
changed = []
removed = []
casing = set()
operator = set()


for fn in diff_fns:
    tdate = datetime.datetime(int(fn[10:14]),int(fn[15:17]),int(fn[18:20]))
    # print(tdate)
    if tdate>= edate:
        download_dates.append(fn[10:20])
        with open(os.path.join(arc_dir,fn),'rb') as f:
            diff_dic = pickle.load(f)
        if len(diff_dic['removed_disc'])>0:
            t = diff_dic['removed_disc'].copy()
            # print(t.head())
            t['date_changed'] = tdate
            t['change_type'] = 'removed'
            removed.append(t)
        if len(diff_dic['added_disc'])>0:
            t = diff_dic['added_disc'].copy()
            t['date_changed'] = tdate
            t['change_type'] = 'added'
            added.append(t)
        if len(diff_dic['changed_disc'])>0:
            t = diff_dic['changed_disc'].copy()
            t['date_changed'] = tdate
            t['change_type'] = 'modified'            
            changed.append(t)
        if len(diff_dic['casing'])>0:
            for item in diff_dic['casing']:
                # print(item)
                if item[1]==None: ig = ''
                else: ig = item[1].strip().lower()
                tup = (item[0],ig)
                casing.add(tup)
        if len(diff_dic['OperatorName'])>0:
            for item in diff_dic['OperatorName']:
                operator.add(item)

alllists = added + changed + removed
wholeset = pd.concat(alllists,sort=True)


In [ ]:
gb_added = wholeset[wholeset.change_type=='added'].groupby('date_changed').size()
added_sum = gb_added.resample("D").sum()
added_sum = added_sum+.001 # to distinguish between zeros and no data

gb_removed = wholeset[wholeset.change_type=='removed'].groupby('date_changed').size()
removed_sum = gb_removed.resample("D").sum()
removed_sum = removed_sum+0.001

gb_changed = wholeset[wholeset.change_type=='modified'].groupby('date_changed').size()
changed_sum = gb_changed.resample("D").sum()
changed_sum = changed_sum+0.001

### set up the difflib outputs

In [ ]:
import difflib
archive_folder = r"D:\openFF_archive\raw_dataframes"
archive_folder = r"C:\MyDocs\integrated\openFF_archive\raw_dataframes"

def get_raw_list():
    alst = os.listdir(archive_folder)
    dates = []
    for f in alst:
        dates.append(f[7:17])
    return dates
    
def get_early_late_fn(change_date,datelst):
    latefn = os.path.join(archive_folder,'raw_df_'+change_date+'.parquet')
    idx = datelst.index(change_date)
    earlyfn = os.path.join(archive_folder,'raw_df_'+datelst[idx-1]+'.parquet')
    # print(earlyfn+'\n'+latefn)
    return earlyfn,latefn

def get_comparison_dfs(apinum,earlyfn, latefn):
    filt = [('APINumber','==',apinum)]
    edf = pd.read_parquet(earlyfn,filters=filt)
    ldf = pd.read_parquet(latefn,filters=filt)
    return edf,ldf
    

In [ ]:
import difflib

def get_difflink_from_df(api,change_date):
    return os.path.join(r"G:\My Drive\webshare\daily_status\diff_files",api+'_'+change_date+'.html')

def get_url_from_df(api,change_date):
    return "https://storage.googleapis.com/open-ff-browser/difflib/",api+'_'+change_date+'.html'

def make_logistics_summary(ddf):
    # assumes single disclosure in ddf
    # start with meta or logistics data, so all is available in any line
    jsd = ddf.JobStartDate.iloc[0]
    jed = ddf.JobEndDate.iloc[0]
    state = ddf.StateName.iloc[0]
    county = ddf.CountyName.iloc[0]
    api = ddf.APINumber.iloc[0]
    operator = ddf.OperatorName.iloc[0]
    wellname = ddf.WellName.iloc[0]
    lat = ddf.Latitude.iloc[0]
    lon = ddf.Longitude.iloc[0]
    datum = ddf.Projection.iloc[0]
    fed = ddf.FederalWell.iloc[0]
    indian = ddf.IndianWell.iloc[0]
    tvd = ddf.TVD.iloc[0]
    tbwv = ddf.TotalBaseWaterVolume.iloc[0]
    tbnwv = ddf.TotalBaseNonWaterVolume.iloc[0]
    did = ddf.DisclosureId.iloc[0]
    ffv = ddf.FFVersion.iloc[0]

    s=  '\n**Logistics Section** \n'
    s+= f'APINumber: {api}\n'
    s+= f'Job Start Date: {jsd}\n'
    s+= f'Job End Date: {jed}\n'
    s+= f'State: {state}\n'
    s+= f'County: {county}\n'
    s+= f'Operator: {operator}\n'
    s+= f'Well Name: {wellname}\n'
    s+= f'Latitude: {lat}  Longitude: {lon}\n'
    s+= f'Projection: {datum}\n'
    s+= f'Federal: {fed}   Indian: {indian}\n'
    s+= f'Total Vertical Depth: {tvd}\n'
    s+= f'Carrier water volume: {tbwv}\n'
    s+= f'Vol of non-water carrier: {tbnwv}\n'
    s+= f'FF Version: {ffv}\n'
    s+= f'DisclosureId: {did}\n'

    return s

def make_products_summary(ddf):
    ###  TradeName products
    ddf.CASNumber = ddf.CASNumber.fillna('--empty--')
    ddf = ddf.sort_values(['TradeName','CASNumber'])
    gb = ddf.groupby('TradeName')['CASNumber'].apply(set).reset_index()
    s= '\n**Products List Section with components**\n'
    for i,row in gb.iterrows():
        s+= f'Name: {row.TradeName}\n'
        lst = list(row.CASNumber)
        lst.sort()
        for cas in lst:
            s+= f'      - {cas}\n'
    return s
    # tn = ddf.TradeName.unique().tolist()
    # tn.sort()
    # for i in tn:
    #     s+= f'Name: {i}\n'

def make_record_count_summary(ddf):
    ###  Number of records
    cn = ddf.CASNumber.notna()&ddf.PercentHFJob.notna()
    s= '\n**Number of records Section**\n'
    s+= f'Total num records: {len(ddf)}\n'
    s+= f'Record with chem info: {len(ddf[cn])}\n'
    return s

def make_num_duplicates(ddf):
    cn = ddf.CASNumber.notna()&ddf.IngredientName.notna()&ddf.PercentHighAdditive.notna()&ddf.PercentHFJob.notna()
    cdup = ddf[['CASNumber','IngredientName','PercentHighAdditive','PercentHFJob','MassIngredient']].duplicated()
    s = '\n**Number of duplicate records**\n'
    t = ddf[cn&cdup].copy()
    s += f'Number of duplicate records: {len(t)}\n'
    # for i,row in t.iterrows():
    #     s += f'duplicated record: {row.CASNumber}, {row.IngredientName}, {row.PercentHFJob}\n'
    return s

def make_chem_list(ddf):
    #  List of UNIQUE CAS/ING
    s = '\n**Unique CASNumber/IngredientName pairs**\n'
    t = ddf.groupby(['CASNumber','IngredientName'],as_index=False).size().rename({'size':'num_recs'},axis=1)
    t = t.sort_values(['CASNumber','IngredientName'])
    for i,row in t.iterrows():
        s += f'CAS: {row.CASNumber}, Name: {row.IngredientName}, How many: {row.num_recs}\n'
    return s

def make_chem_data_list(ddf):
    # list of main components of a data record - only those with CASNumber or IngredientName
    s = '\n**record data**\n\n'
    sortby_order = ['TradeName','Purpose','Supplier','PercentHFJob','CASNumber','IngredientName']
    # t = ddf.groupby(['CASNumber','IngredientName'],as_index=False).size().rename({'size':'num_recs'},axis=1)
    t = ddf.sort_values(sortby_order)
    for i,row in t.iterrows():
        s += f'{row.TradeName};{row.Purpose};{row.Supplier}\n\t{row.CASNumber}\n\t{row.IngredientName}\n\t{row.PercentHighAdditive};{row.PercentHFJob};{row.MassIngredient}\n'
    return s
    
# def make_diff_str(str1,str2,section='logistics'):
#     diff = difflib.context_diff(str1, str2,n=20,fromfile='', tofile='')
#     s = f'\n\n<<{section}>>\n'
#     for line in diff:
#         s+= line
#     # print(f'*****   DIFF FILE FOLLOWS *****\n{s}')
#     return s

def make_html_diff(str1,str2):
    HD = difflib.HtmlDiff(wrapcolumn=80)
    return HD.make_file(str1, str2,fromdesc='original', todesc='modified')

def get_diff_size(str1,str2):
    diff = difflib.context_diff(str1, str2,fromfile='original', tofile='modified')
    s = ''
    for line in diff:
        s+= line
    return len(s)

# def comparison_alerts(edf,ldf):
    
    
def PDF_like_compare(apinumber,change_date,datelst,outhtml):
    print(f'working on {apinumber} {change_date}')
    earlyfn,latefn = get_early_late_fn(change_date=change_date,datelst=datelst)
    edf,ldf = get_comparison_dfs(apinumber,earlyfn,latefn)
    str1 = f'Archive file :\n\t{earlyfn}\n'
    str1 += make_logistics_summary(edf)
    str1 += make_products_summary(edf)
    str1 += make_record_count_summary(edf)
    str1 += make_num_duplicates(edf)
    str1 += make_chem_list(edf)
    str1 += make_chem_data_list(edf)
    lst1 = str1.split('\n')
    str2 = f'Archive file :\n\t{latefn}\n'
    str2 += make_logistics_summary(ldf)
    str2 += make_products_summary(ldf)
    str2 += make_record_count_summary(ldf)
    str2 += make_num_duplicates(ldf)
    str2 += make_chem_list(ldf)
    str2 +=make_chem_data_list(ldf)
    lst2 = str2.split('\n')
    # print(str1)
    out  = make_html_diff(lst1,lst2)
    # outhtml = get_difflink_from_df(apinumber)
    with open(outhtml,'w') as f:
        f.write(out)
    return(get_diff_size(lst1,lst2))
    

# PDF_like_compare(edf,ldf,apinum)

## Downloads from FracFocus


In [ ]:
wholeset['job_end_date'] = wholeset.JobEndDate.str.split().str[0]
wholeset['job_end_date'] = pd.to_datetime(wholeset.job_end_date,format="%m/%d/%Y")
wholeset['FF_disc'] = wholeset.apply(lambda x: th.getFFLink(x), axis=1)
wholeset['disc_link'] = wholeset.apply(lambda x: th.getDisclosureLink(APINumber=x.APINumber,
                                                                      disclosureid=x.discID,
                                                                      text_to_show='Open-FF disclosure',
                                                                      use_remote=True,
                                                                      check_if_exists=False), axis=1)
cond = (wholeset.date_changed>=last_repo)&(wholeset.change_type!='removed')
wholeset.disc_link = np.where(cond,' ',wholeset.disc_link)
wholeset['TBWV'] = wholeset.TotalBaseWaterVolume.astype('float64')
wholeset['TBWV'] = wholeset.TBWV.map(lambda x: th.round_sig(x,5))
wholeset['TBWV'] = wholeset.TBWV.fillna(' - ')

In [ ]:
wholeset.change_type.value_counts()


In [ ]:
wholeset['has more than one'] = np.where(wholeset.APINumber.duplicated(keep=False),'APINumber dupe','')

In [ ]:
# for curated disclosures, get detected flaws

DiDs = wholeset.discID.unique().tolist()
rec_iss = pd.read_parquet(os.path.join(hndl.curr_repo_dir,'record_issues.parquet'),columns=['r_flags','reckey'])
reck = pd.read_parquet(os.path.join(hndl.curr_repo_pkl_dir,'chemrecs.parquet'),columns=['DisclosureId','reckey'])
mg = pd.merge(rec_iss,reck,on='reckey',how='left')
gb = mg[mg.DisclosureId.isin(DiDs)].groupby('DisclosureId',as_index=False)['r_flags'].apply(set)

def str_from_set(x):
    s = ''
    for item in x:
        s += item +' '
    return s

gb['recstr'] = gb.r_flags.map(lambda x: str_from_set(x))

In [ ]:
dis_iss = pd.read_parquet(os.path.join(hndl.curr_repo_dir,'disclosure_issues.parquet'),columns=['d_flags','DisclosureId'])
dis_iss = dis_iss[dis_iss.DisclosureId.isin(DiDs)]
gb = gb.merge(dis_iss,on='DisclosureId',how='outer')
gb = gb.fillna('')
gb['issues'] = gb.d_flags +' '+gb.recstr
wholeset = wholeset.merge(gb[['DisclosureId','issues']],left_on='discID',right_on='DisclosureId',how='left')
wholeset.issues = wholeset.issues.fillna('')

## watch list summary
See bottom of page for whole list

In [ ]:
# url = 'https://raw.githubusercontent.com/gwallison/FF_issues/master/watch_list.csv'
# wdf = pd.read_csv(url,dtype = {'APINumber':'str'})
url = 'https://raw.githubusercontent.com/gwallison/FF_issues/master/watch_list_master.parquet'
wdf = pd.read_parquet(url)
wdf = wdf.rename({'DisclosureId':'wl_DisclosureId'},axis=1)
# wdf.date_entered = pd.to_datetime(wdf.date_entered,format='%m/%d/%y')
# wdf.FF_report_date = pd.to_datetime(wdf.FF_report_date,format='%m/%d/%y')
# wdf.Blog_date = pd.to_datetime(wdf.Blog_date,format='%m/%d/%y')
# wdf.FF_updates = pd.to_datetime(wdf.FF_updates,format='%m/%d/%y')

apis = wdf.APINumber.unique().tolist()

watchlist_found = pd.merge(wdf,wholeset,on='APINumber',how='inner')
watchlist_found[['wl_name','change_type']].value_counts()

## Pattern of new disclosure additions
These disclosures are detected as new because their `DisclosureId` number hasn't been in the database before.  Note that it is possible that they are a new version of a previously published disclosure; sometimes operators change disclosures by removing the old one from FracFocus and creating a new one.

In [ ]:
# print(download_dates)
import warnings
warnings.filterwarnings("ignore")
import calplot
calplot.calplot(added_sum,  cmap='Spectral_r');


- **Blue line** = Cumulative new disclosures added (my include replacements for removed disclosures)
- **Orange line** = New disclosures with detected issues
- **Vertical dashed line** = date of last Open-FF data set generation

In [ ]:
t = wholeset[wholeset.change_type=='added']

gb = t.groupby('date_changed',as_index=False).size()
gb['cs'] = gb['size'].cumsum()
gb = gb[['date_changed','cs']].set_index('date_changed')
ax = gb.cs.plot(title='Number of new disclosures',
               ylabel='cumulative disclosures', xlabel='date changed')

gb = t[t.issues.str.len()>1].groupby('date_changed',as_index=False).size()
gb['with_issues'] = gb['size'].cumsum()
gb = gb[['date_changed','with_issues']].set_index('date_changed')
ax = gb.with_issues.plot(ax=ax)


ax.axvline(last_repo, color="green", linestyle="dashed");


In [ ]:
show_whole = wholeset[wholeset.change_type=='added'].rename({'job_end_date':'job end date','date_changed':'date added',
                            'change_type':'change type'},axis=1)
show_whole[['APINumber','FF_disc','disc_link','job end date','StateName','CountyName','OperatorName',
          'TBWV','date added','has more than one','issues']].to_parquet(added_disc_fn)

show_whole[['FF_disc','disc_link','job end date','StateName','CountyName','OperatorName',
          'TBWV','date added','has more than one','issues']].reset_index(drop=True)


### Removed disclosures

In [ ]:
if removed_sum.sum()>0.5:
    calplot.calplot(removed_sum, cmap='Spectral_r');
else:
    display(md('#### No removed disclosures found'))

In [64]:
# from: https://www.geeksforgeeks.org/test-the-given-page-is-found-or-not-on-the-server-using-python/#
import requests
def url_ok(url):
	# exception block
	try:
		# pass the url into
		# request.head
		response = requests.head(url)
		
		# check the status code
		if response.status_code == 200:
			return True
		else:
			return False
	except requests.ConnectionError as e:
		return e


In [ ]:
datelst = get_raw_list()
old_rem = pd.read_parquet(removed_disc_fn)

if removed_sum.sum()>0.5:   #0.5 to account for the 0.001 baseline
    show_whole = wholeset[wholeset.change_type=='removed'].rename({'job_end_date':'job end date',
                                                                   'date_changed':'date removed',
                                                                   'change_type':'change type'},axis=1)
    # show_whole['diff_fn'] = show_whole.APINumber.map(lambda x: get_difflink_from_df(x))
    ldiffs = []
    hmto = []
    for i,row in show_whole.iterrows():
        if row['has more than one']:
            chg_date = row['date removed'].strftime("%Y-%m-%d")
            outfn = get_difflink_from_df(row.APINumber,chg_date)
            url = get_url_from_df(row.APINumber,chg_date)
            if url_ok(url):  # don't remake this diff file if already uploaded
                try: 
                    ldiff_df = old_rem[(old_rem.APINumber==row.APINumber) & (old_rem['date removed']==row['date removed'])]['difflib size']
                    if len(ldiff_df)>0: # the old df is usable
                        ldiffs.append(ldiff_df.iloc[0])
                        hmto.append(th.wrap_URL_in_html(outfn,'compare with new'))
                        continue
                    else:
                        print(' -- regenerating...') # let it regenerate
                except:
                    print(f'something went wrong with already generated {row.APINumber}') 

            # make the diff file
            try:
                len_diff = PDF_like_compare(row.APINumber,chg_date,datelst,outhtml=outfn)
                ldiffs.append(len_diff)
                hmto.append(th.wrap_URL_in_html(url,'compare with new'))
            except:
                ldiffs.append(-1)
                hmto.append('error with diff process')
        else:
            ldiffs.append(-1)
            hmto.append('')
    show_whole['difflib size'] = ldiffs
    show_whole['has more than one'] = hmto
    show_whole.TBWV = show_whole.TBWV.astype('str')
    show_whole['difflib size'] = show_whole['difflib size'].astype('str')

    show_whole[['APINumber','FF_disc','disc_link','job end date','StateName','CountyName','OperatorName',
          'TBWV','date removed','has more than one','difflib size','issues']].to_parquet(removed_disc_fn)
    
    iShow(show_whole[['FF_disc','disc_link','job end date','StateName','CountyName','OperatorName',
          'TBWV','date removed','has more than one','difflib size','issues']].reset_index(drop=True))

### Modified disclosures

In [ ]:
if changed_sum.sum()>0.5:  #0.5 to account for the 0.001 baseline
    calplot.calplot(changed_sum, cmap='Spectral_r');
else:
    display(md('#### No modified disclosures detected'))

In [ ]:
if changed_sum.sum()>0.5:
    show_whole = wholeset[wholeset.change_type=='modified'].rename({'job_end_date':'job end date','date_changed':'date modified',
                            'change_type':'change type'},axis=1)
    show_whole[['APINumber','FF_disc','disc_link','job end date','StateName','CountyName','OperatorName',
          'TBWV','date modified','has more than one','issues']].to_parquet(modified_disc_fn)
    iShow(show_whole[['FF_disc','disc_link','job end date','StateName','CountyName','OperatorName',
          'TBWV','date modified','has more than one','issues']].reset_index(drop=True))

In [ ]:
# get list of *reported* issues
def add_to_set(s, iset):
    lst = s.split()
    for i in lst:
        iset.add(i)
    return iset

iset = set()
for i, row in wholeset.iterrows():
    iset = add_to_set(row.issues,iset)

### Issues list

In [ ]:
import FF_issues.process_master_files as pmf
pobj = pmf.Process_Master_Files()
df = pobj.process_obj()


#### Discloure-level Issues

In [ ]:
c = df.Flag_id.str[0]=='d'
c1 = df.Flag_id.isin(iset)
t = df[c&c1].copy()
t['flaw_link'] = t.Flag_id.map(lambda x: th.getFlawLink(x))
t[['Title','flaw_link','Warning_level']]

#### Record-level Issues

In [ ]:
c = df.Flag_id.str[0]=='r'
# c1 = df.Flag_id.isin(iset)
t = df[c&c1].copy()
t['flaw_link'] = t.Flag_id.map(lambda x: th.getFlawLink(x))
t[['Title','flaw_link','Warning_level']]

## New CASNumber : IngredientName pairs

In [ ]:
# Are there new casing?
repo_casing = fh.get_casing_df()
changed_casing = pd.DataFrame(casing,columns=['CASNumber','IngredientName'])
mg = pd.merge(changed_casing,repo_casing,on=['CASNumber','IngredientName'], how='left',indicator=True)
mg[mg._merge=='left_only'][['CASNumber','IngredientName']]

In [ ]:
# Are there new Operators?
repo_companies = fh.get_company_df()
complst = repo_companies.rawName.tolist()
newcomp = []
for op in operator:
    if not op in complst:
        newcomp.append(op)
if len(newcomp)> 0:
    display(md('## New Operator names detected'))
    newcomp.sort()
    for item in newcomp:
        display(md(f'##### {item}'))
else:
    display(md('### No new operator names detected'))

___
## Watch list
Wells with changed disclosures that were previously detected with problems

In [ ]:
watchlist_found

## Monitor ECMC disclosure list

In [ ]:
url = "https://ecmc.state.co.us/depot/Stats/ChemicalDisclosures/Index/Locations"
t = pd.read_html(url)
t[0]

## Monitoring sand-dominated disclosures

The following code is used to keep a running list (in a dataframe) of the disclosure that have greater than 50% sand.  This list will be used to monitor the success of efforts to inform companies of the problem.

In [ ]:
import openFF.build.core.Bulk_data_reader as bdr
rff = bdr.Read_FF()

In [ ]:
import os
import datetime
rawdir = r"C:\MyDocs\integrated\openFF_archive\raw_dataframes"
dirlist = os.listdir(rawdir)
lastraw = dirlist[-1]
print(f'Using {lastraw} as data source')
rawdf = pd.read_parquet(os.path.join(rawdir,lastraw))
rawdate = datetime.datetime(int(lastraw[7:11]),int(lastraw[12:14]),int(lastraw[15:17]))
# print(rawdate)

In [ ]:
rawdf.PercentHFJob = pd.to_numeric(rawdf.PercentHFJob)
rawdf = rff.make_date_fields(rawdf)
gb = rawdf[rawdf.CASNumber=='14808-60-7'].groupby('DisclosureId',as_index=False).PercentHFJob.sum()
gb2 = rawdf.groupby('DisclosureId',as_index=False)[['APINumber','OperatorName','StateName',
                                                    'CountyName','JobEndDate','date','year']].first()
mg = pd.merge(gb,gb2,on='DisclosureId',how='inner')

sand_disc = mg[mg.PercentHFJob>50].DisclosureId.tolist()

In [ ]:
fn = r"G:\My Drive\webshare\daily_status\sand_monitoring_df.parquet"
try:
    master = pd.read_parquet(fn)
except:
    master = pd.DataFrame({'DisclosureId':[],'added_date':[],'removed_date':[]})
together = pd.merge(master,pd.DataFrame({'DisclosureId':sand_disc}),
                    on='DisclosureId',how='outer',indicator=True)
# now evaluate for new and removed
removed = together[together._merge=='left_only'].DisclosureId.tolist()
added = together[together._merge=='right_only'].DisclosureId.tolist()

# For removed, make "removed_date" equal to rawdate
c1 = master.DisclosureId.isin(removed)
c2 = master.removed_date.isna()
print(f'Number of disclosures removed from sand dominated list: {(c1&c2).sum()}')
master['removed_date'] = np.where(c1&c2,rawdate,master.removed_date)

# For added, get the other columns from mg and concatenate, then put in date_added
added_df = mg[mg.DisclosureId.isin(added)].copy()
print(f'Number of disclosures added as sand-dominated: {len(added_df)}')
added_df['added_date'] = rawdate
condf = pd.concat([master,added_df],sort=True)
condf.to_parquet(fn)
c3 = condf.removed_date.isna()
print(f'Total number of sand dominated disclosures still on the list: {len(condf[c3])}')
print(f'Updated file saved to {fn}')
